#Bibliotecas

In [1]:
import cv2
import numpy as np
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, jaccard_score,f1_score
from skimage import io, segmentation, measure, filters
from skimage.color import label2rgb
import pandas as pd
from skimage.feature import canny
from skimage.transform import hough_line
from skimage import metrics
from sklearn.metrics import precision_score




#Diretório das imagens processadas

In [2]:
imagens_dataset_entrada='/content/drive/MyDrive/Projeto_final/dataset/saida_imagens_cinza_sobel'
imagens_dataset_saida='/content/drive/MyDrive/Projeto_final/dataset/Resultado_Metricas'

In [3]:


# Lista para armazenar as métricas de cada imagem
df_metricas = pd.DataFrame(columns=['Precisão', 'Recall', 'YoU','F1-Score'])

if not os.path.exists(imagens_dataset_saida):
    os.makedirs(imagens_dataset_saida)

for nome_imagem in os.listdir(imagens_dataset_entrada):
    # Verifica se o arquivo é uma imagem
    if nome_imagem.endswith(".jpg") or nome_imagem.endswith(".jpeg") or nome_imagem.endswith(".png"):
        # Carrega a imagem em escala de cinza
        imagem = io.imread(os.path.join(imagens_dataset_entrada, nome_imagem), as_gray=True)
        
        # Binariza a imagem com método de Otsu
        # Obtém o limiar com a função threshold_otsu
        limiar_otsu = filters.threshold_otsu(imagem)

        # Cria um array com o mesmo valor do limiar
        limiar_array = np.full(imagem.shape, limiar_otsu)

        # Binariza a imagem utilizando o array de limiar
        imagem_binarizada = imagem > limiar_array

        
        
        # Segmentação da imagem usando o filtro Sobel e o algoritmo de watershed
        marcadores = filters.sobel(imagem_binarizada)
        segmentacao = segmentation.watershed(marcadores, markers=None, mask=imagem_binarizada)

        # Criação dos rótulos verdadeiros
        rotulos_preditos, rotulos_verdadeiros = measure.label(segmentacao, return_num=True)

                  

        # Calcula as métricas de avaliação de segmentação
        rotulos_verdadeiros = io.imread(os.path.join(imagens_dataset_entrada, nome_imagem), as_gray=True)
        rotulos_verdadeiros = measure.label(rotulos_verdadeiros)
        rotulos_preditos = measure.label(segmentacao)
      

        #matriz de YoU
        YoU_matriz=jaccard_score(rotulos_verdadeiros.ravel(),rotulos_preditos.ravel(),average=None)
        #selecionar apenas pares com YoU maior que 50

        indice_achado=np.where(YoU_matriz >0.5)[0]
        rotulos_verdadeiros_bin_achados=rotulos_verdadeiros[:,indice_achado]
        rotulos_preditos_bin_achados=rotulos_preditos[:,indice_achado]

        # Converter as labels para o formato binário
        mlb = MultiLabelBinarizer()
        rotulos_verdadeiros_bin_achados = mlb.fit_transform(rotulos_verdadeiros_bin_achados)
        rotulos_preditos_bin_achados = mlb.transform(rotulos_preditos_bin_achados)
        # Verifica se há valores desconhecidos
        nan_valores = np.isnan(rotulos_verdadeiros_bin_achados).any() or np.isnan(rotulos_preditos_bin_achados).any()

        if nan_valores:
          # Remove valores desconhecidos
          rotulos_verdadeiros_bin_achados = np.nan_to_num(rotulos_verdadeiros_bin_achados)
          rotulos_preditos_bin_achados = np.nan_to_num(rotulos_preditos_bin_achados)


      # Calcular as métricas utilizando as labels binárias
        precisao = precision_score(rotulos_verdadeiros_bin_achados, rotulos_preditos_bin_achados, average='weighted',zero_division=0)
        reca = recall_score(rotulos_verdadeiros_bin_achados, rotulos_preditos_bin_achados, average='weighted',zero_division=0)
        iou = jaccard_score(rotulos_verdadeiros_bin_achados, rotulos_preditos_bin_achados, average='weighted', zero_division=0)
        f1_s = f1_score(rotulos_verdadeiros_bin_achados, rotulos_preditos_bin_achados, average='weighted', zero_division=0)

        # Salva a imagem de saída com a segmentação aplicada
        #segmentacao = segmentacao.astype('float32')
        #segmentacao = (segmentacao * 255).astype(np.uint8)

        io.imsave(os.path.join(imagens_dataset_saida, nome_imagem), (label2rgb(segmentacao) * 255).astype(np.uint8))

    # Adiciona as métricas da imagem ao DataFrame

        # Adiciona as métricas da imagem ao DataFrame
        #df_metricas.loc[nome_imagem] = [precisao, reca, iou]
        df_metricas = df_metricas.append({'Imagem': nome_imagem, 'Precisão': precisao, 'Recall': reca, 'YoU': iou,'F1_score':f1_s}, ignore_index=True)





        print(f"Imagem: {nome_imagem}")
        print(f"Precisão: {precisao:.2f}")
        print(f"Recall: {reca:.2f}")
        print(f"YOU: {iou:.2f}")
        print(f"F1_score: {f1_s:.2f}")
# Calcula as médias das métricas de todas as imagens


medias_metricas = df_metricas.mean()



# Exibe as métricas médias
print('Métricas médias:')
print('-----------------')
print(f"Precisão: {medias_metricas['Precisão']:.2f}")
print(f"Revocação: {medias_metricas['Recall']:.2f}")
print(f"YoU: {medias_metricas['YoU']:.2f}")
print(f"F1_score: {medias_metricas['F1_score']:.2f}")
print(type(imagem_binarizada))
        

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [106, 212, 221, 223, 279, 288, 348, 352, 399, 402, 403, 409, 413, 419, 421, 424, 438, 446, 482, 489, 49, 504, 517, 523, 542, 557, 581, 585, 59, 593, 625, 638, 711, 74, 745, 760, 85, 99] will be ignored
  warnings.warn(
<ipython-input-3-800ca82d1755>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metricas = df_metricas.append({'Imagem': nome_imagem, 'Precisão': precisao, 'Recall': reca, 'YoU': iou,'F1_score':f1_s}, ignore_index=True)


Imagem: image-46_sobel.png
Precisão: 0.68
Recall: 0.68
YOU: 0.68
F1_score: 0.68


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [131, 391, 401] will be ignored
  warnings.warn(
<ipython-input-3-800ca82d1755>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metricas = df_metricas.append({'Imagem': nome_imagem, 'Precisão': precisao, 'Recall': reca, 'YoU': iou,'F1_score':f1_s}, ignore_index=True)


Imagem: image-93_sobel.png
Precisão: 0.94
Recall: 0.94
YOU: 0.94
F1_score: 0.94


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [107, 19, 440, 460, 62, 68, 787] will be ignored
  warnings.warn(
<ipython-input-3-800ca82d1755>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metricas = df_metricas.append({'Imagem': nome_imagem, 'Precisão': precisao, 'Recall': reca, 'YoU': iou,'F1_score':f1_s}, ignore_index=True)


...

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [106, 167, 170, 182, 194, 412, 465, 480, 498, 564, 585, 645, 692, 699, 725] will be ignored
  warnings.warn(
<ipython-input-3-800ca82d1755>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metricas = df_metricas.append({'Imagem': nome_imagem, 'Precisão': precisao, 'Recall': reca, 'YoU': iou,'F1_score':f1_s}, ignore_index=True)


Imagem: image-43_sobel.png
Precisão: 0.85
Recall: 0.85
YOU: 0.85
F1_score: 0.85


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [304, 324, 370, 374] will be ignored
  warnings.warn(


Imagem: image-89_sobel.png
Precisão: 0.94
Recall: 0.94
YOU: 0.94
F1_score: 0.94
Métricas médias:
-----------------
Precisão: 0.90
Revocação: 0.90
YoU: 0.90
F1_score: 0.90
<class 'numpy.ndarray'>


<ipython-input-3-800ca82d1755>:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metricas = df_metricas.append({'Imagem': nome_imagem, 'Precisão': precisao, 'Recall': reca, 'YoU': iou,'F1_score':f1_s}, ignore_index=True)
<ipython-input-3-800ca82d1755>:91: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  medias_metricas = df_metricas.mean()
